<a href="https://colab.research.google.com/github/alex-jk/GTA_apartment_rentals/blob/main/code/gta_apartment_rentals_get_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread google-auth google-auth-oauthlib google-auth-httplib2
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.9/186.9 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.1
    Uninstalling google-auth-httplib2-0.1.1:
      Successfully uninstalled google-auth-httplib2-0.1.1
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.28.1 which is incompatible.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [1]:
import pandas as pd
import requests
import re
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from google.colab import drive
drive.mount('/content/drive')

credentials, project = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])

Mounted at /content/drive


#### Import input data GTA_apartment_buildings_list

In [4]:
import gspread

gc = gspread.authorize(credentials)

sheet_name = 'GTA_apartment_buildings_list'
sh = gc.open(sheet_name)
worksheet = sh.sheet1

# Get all the records of the data
apartments_data = worksheet.get_all_records()

# Convert to a DataFrame
apartments_data_df = pd.DataFrame(apartments_data)

# Display the first few rows of the dataframe
print(f"\n------------ Apartments data df shape: {apartments_data_df.shape}")
print("\n------------")
print(apartments_data_df.head())


------------ Apartments data df shape: (117, 7)

------------
                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   

                                       phone_numbers    Elementary_School  \
0                         437-973-3626, 416-665-4430          Rockford PS   
1           437-900-6800, 416-665-2100, 437-900-6811          Rockford PS   
2                         416-485-7207, 416-751-3368    Oriole Park Jr PS   
3           888-244-6696, 647-507-4461, 416-487-4461     Davisville Jr PS   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...  Ledbury Park E & MS   

         Middle_School           High_School street_name  
0    Fisherville Sr PS  Northview Heights SS     Antibes  
1    Fisherville Sr

In [32]:
def find_phone_number(address, query_str):
    query = address.replace(' ', '+') + query_str
    url = f"https://www.google.com/search?q={query}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Use regular expressions to find phone numbers
        phone_regex = re.compile(r'\(\d{3}\)\s*\d{3}-\d{4}|\d{3}-\d{3}-\d{4}')
        text = soup.get_text()
        # print("\n----------------")
        # print(text)
        phone_numbers = phone_regex.findall(text)

        if phone_numbers:
            # Remove duplicates and format phone numbers consistently
            formatted_numbers = [re.sub(r'[^0-9]', '', number) for number in set(phone_numbers)]
            formatted_numbers = [f"{num[:3]}-{num[3:6]}-{num[6:]}" for num in formatted_numbers]
            formatted_numbers = list(set(formatted_numbers))

            return formatted_numbers
        else:
            return []
    else:
        return []

In [33]:
def expand_ranges(beds_list):
    expanded_list = []
    for item in beds_list:
        if '-' in item:  # Check if item is a range
            start, end = map(int, item.split('-'))
            expanded_list.extend(range(start, end + 1))  # Add all numbers in the range
        else:
            expanded_list.append(int(item))  # Add individual numbers directly
    return expanded_list

def find_num_beds(address, query_str):
    query = address.replace(' ', '+') + query_str
    query_apartments_com = address.replace(' ', '+') + "+apartments+com"
    url = f"https://www.google.com/search?q={query}"
    url_apartments_com = f"https://www.google.com/search?q={query_apartments_com}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    text_numbers_to_int = {'one': 1, '1': 1, 'two': 2, '2': 2, 'three': 3, '3': 3, 'four': 4, 'five': 5}

    response = requests.get(url, headers=headers)
    response_apartments_com = requests.get(url_apartments_com, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Use regular expressions to find phone numbers
        numeric_bed_regex = re.compile(r'\b(\d+(?:-\d+)?)\s*beds?\b', re.IGNORECASE)
        text_bed_regex = re.compile(r'\b(one|two|three|four|five|1|2|3)\s*bedroom?\b', re.IGNORECASE)
        text = soup.get_text()
        # print("\n----------------")
        # print(text)

        numeric_beds_list = numeric_bed_regex.findall(text)
        expanded_beds_list = list(set( expand_ranges(numeric_beds_list) ))
        expanded_beds_list.sort()
        # print(f"\n numeric beds: {expanded_beds_list}")

        text_beds_list = text_bed_regex.findall(text)
        text_beds_list = list(set(text_beds_list))
        text_beds_list = [text_numbers_to_int[bed.lower()] for bed in text_beds_list]
        if len(text_beds_list) > 0:
          text_beds_list = list(range(1, max(text_beds_list) + 1))
        # print(f"\n text beds: {text_beds_list}")

        combined_set = set(expanded_beds_list + text_beds_list)
        combined_set = sorted(list(combined_set))

        # if response_apartments_com.status_code == 200:
        #   soup_apartments_com = BeautifulSoup(response_apartments_com.text, 'html.parser')
        #   links = soup.find_all('a')
        #   # Attempt to find the first valid apartments.com URL
        #   apartments_url = None
        #   for link in links:
        #       href = link.get('href')
        #       if 'apartments.com' in href:
        #           # Typical Google search result URLs are redirected, so parse them to get the actual URL
        #           start = href.find('http')
        #           end = href.find('&', start)
        #           if start != -1 and end != -1:
        #               apartments_url = href[start:end]
        #               break

        #   if apartments_url:
        #       print("Found Apartments.com URL:", apartments_url)
        #   else:
        #       print("No Apartments.com URL found in the first few search results.")

        if combined_set:
            smallest_bed_number = combined_set[0]
            complete_beds_list = list(range(1, smallest_bed_number)) + combined_set

            # Remove duplicates and sort again if necessary
            complete_beds_list = sorted(set(complete_beds_list))
            complete_beds_str = ', '.join(map(str, complete_beds_list))
            return complete_beds_str
        else:
            return []
    else:
        return []

In [34]:
# def conditionally_find_num_beds(row, query_str):
#     # If num_beds is missing, then call find_num_beds
#     if pd.isna(row['num_beds']) or not str(row['num_beds']).strip():
#         return find_num_beds(row['apartment_address'], query_str)
#     else:
#         return row['num_beds']

def conditionally_find_value(row, column_name, search_function, *args, **kwargs):
    """
    Checks if the specified column in a DataFrame row is missing or empty. If so, it calls
    a specified search function to find the value.

    Parameters:
    - row: The row of the DataFrame.
    - column_name: The name of the column to check and potentially fill.
    - search_function: The function to call if the column value is missing or empty.
                       This function should accept the apartment address as its first argument,
                       followed by any *args and **kwargs necessary for the search.
    - *args: Additional positional arguments to pass to the search_function.
    - **kwargs: Additional keyword arguments to pass to the search_function.

    Returns:
    - The original value from the row[column_name] if it exists and is not empty,
      otherwise the result from search_function.
    """
    # Check if the column value is missing or empty
    if pd.isna(row[column_name]) or not str(row[column_name]).strip():
        # Call the search function with the address, args, and kwargs
        return search_function(row['apartment_address'], *args, **kwargs)
    else:
        # Return the original value
        return row[column_name]

In [35]:
ind = 0

print(apartments_data_df['apartment_address'][ind])
print( find_phone_number(apartments_data_df['apartment_address'][ind], "+apartment+building+phone+number") )

print( find_num_beds(apartments_data_df['apartment_address'][ind], "+apartment+rent+beds") )

1 Fountainhead Rd, North York, ON M3J 1V6
['416-663-7578', '416-663-4444']
1, 2


In [36]:
# Apply the find_phone_numbers function to each row
query_str = "+apartment+building+phone+number"

tqdm.pandas()
# apartments_data_df['phone_numbers'] = apartments_data_df['apartment_address'].progress_apply(lambda address: find_phone_number(address, query_str))
apartments_data_df['phone_numbers'] = apartments_data_df.progress_apply(conditionally_find_value, axis=1,
                                                                        column_name='phone_numbers',
                                                                        search_function=find_phone_number,
                                                                        query_str=query_str)

print(apartments_data_df.head(10))

100%|██████████| 117/117 [00:00<00:00, 22152.01it/s]

                              apartment_address num_beds  \
0     1 Fountainhead Rd, North York, ON M3J 1V6  1, 2, 3   
1     10 Wilmington Ave, North York, ON M3H 5H7     1, 2   
2  1042 Sheppard Avenue W., Toronto, ON M3H 2T6     1, 2   
3       11 Goldfinch Ct, North York, ON M2R 2C2  1, 2, 3   
4      110 Inverlochy Blvd, Markham, ON L3T 3R6  1, 2, 3   
5      111 Inverlochy Blvd, Markham, ON L3T 3R7  1, 2, 3   
6       1130 Wilson Avenue, Toronto, ON M3M 1K3  1, 2, 3   
7          12 Goldfinch Ct, Toronto, ON M2R 2C4  1, 2, 3   
8      12 Oakburn Crescent, Toronto, ON M2N 2T4     1, 2   
9         12 Rockford Road, Toronto, ON M2R 3A2  1, 2, 3   

                                       phone_numbers Elementary_School  \
0                         416-663-7578, 416-663-4444                     
1                         253-256-1634, 437-500-1353                     
2                                       416-636-9130                     
3  416-633-8409, 647-701-9936, 416-630-3553

In [37]:
# Apply the find_phone_numbers function to each row
query_str = "+apartment+rent+beds"

# apartments_data_df['num_beds'] = apartments_data_df.progress_apply(lambda row: conditionally_find_num_beds(row, query_str), axis=1)
apartments_data_df['num_beds'] = apartments_data_df.progress_apply(conditionally_find_value, axis=1,
                                                                        column_name='num_beds',
                                                                        search_function=find_num_beds,
                                                                        query_str=query_str)

print(apartments_data_df.head(10))

100%|██████████| 117/117 [00:00<00:00, 13386.44it/s]

                              apartment_address num_beds  \
0     1 Fountainhead Rd, North York, ON M3J 1V6  1, 2, 3   
1     10 Wilmington Ave, North York, ON M3H 5H7     1, 2   
2  1042 Sheppard Avenue W., Toronto, ON M3H 2T6     1, 2   
3       11 Goldfinch Ct, North York, ON M2R 2C2  1, 2, 3   
4      110 Inverlochy Blvd, Markham, ON L3T 3R6  1, 2, 3   
5      111 Inverlochy Blvd, Markham, ON L3T 3R7  1, 2, 3   
6       1130 Wilson Avenue, Toronto, ON M3M 1K3  1, 2, 3   
7          12 Goldfinch Ct, Toronto, ON M2R 2C4  1, 2, 3   
8      12 Oakburn Crescent, Toronto, ON M2N 2T4     1, 2   
9         12 Rockford Road, Toronto, ON M2R 3A2  1, 2, 3   

                                       phone_numbers Elementary_School  \
0                         416-663-7578, 416-663-4444                     
1                         253-256-1634, 437-500-1353                     
2                                       416-636-9130                     
3  416-633-8409, 647-701-9936, 416-630-3553

In [38]:
print(apartments_data_df.head(10))

                              apartment_address num_beds  \
0     1 Fountainhead Rd, North York, ON M3J 1V6  1, 2, 3   
1     10 Wilmington Ave, North York, ON M3H 5H7     1, 2   
2  1042 Sheppard Avenue W., Toronto, ON M3H 2T6     1, 2   
3       11 Goldfinch Ct, North York, ON M2R 2C2  1, 2, 3   
4      110 Inverlochy Blvd, Markham, ON L3T 3R6  1, 2, 3   
5      111 Inverlochy Blvd, Markham, ON L3T 3R7  1, 2, 3   
6       1130 Wilson Avenue, Toronto, ON M3M 1K3  1, 2, 3   
7          12 Goldfinch Ct, Toronto, ON M2R 2C4  1, 2, 3   
8      12 Oakburn Crescent, Toronto, ON M2N 2T4     1, 2   
9         12 Rockford Road, Toronto, ON M2R 3A2  1, 2, 3   

                                       phone_numbers Elementary_School  \
0                         416-663-7578, 416-663-4444                     
1                         253-256-1634, 437-500-1353                     
2                                       416-636-9130                     
3  416-633-8409, 647-701-9936, 416-630-3553

In [48]:
def extract_street_name(address):
    match = re.search(r'\d+\s([A-Za-z\s]+)\s', address)
    if match:
        return match.group(1).strip()
    else:
        return None

ind = 7
print( apartments_data_df['apartment_address'].iloc[ind] )

extract_street_name( apartments_data_df['apartment_address'].iloc[ind] )

apartments_data_df['street_name'] = apartments_data_df['apartment_address'].apply(extract_street_name)

12 Goldfinch Ct, Toronto, ON M2R 2C4


In [49]:
apartments_data_df['num_beds'] = apartments_data_df['num_beds'].astype(str)

In [50]:
from gspread_dataframe import set_with_dataframe

spreadsheet_title = 'GTA_apartment_buildings_output'

try:
    # Attempt to open the existing spreadsheet
    sh = gc.open(spreadsheet_title)
    # If successful, delete the spreadsheet by moving it to trash
    gc.del_spreadsheet(sh.id)
    print(f"Existing spreadsheet '{spreadsheet_title}' moved to trash.")
except gspread.SpreadsheetNotFound:
    # If the spreadsheet does not exist, just continue
    print(f"No existing spreadsheet named '{spreadsheet_title}' found. Creating a new one.")

sh = gc.create(spreadsheet_title)
worksheet = sh.get_worksheet(0)

set_with_dataframe(worksheet, apartments_data_df)

Existing spreadsheet 'GTA_apartment_buildings_output' moved to trash.


#### Import output dataset

In [12]:
sheet_name = 'GTA_apartment_buildings_output'
sh = gc.open(sheet_name)
worksheet = sh.sheet1

# Get all the records of the data
apartments_data_out = worksheet.get_all_records()

# Convert to a DataFrame
apartments_data_out_df = pd.DataFrame(apartments_data_out)

# create address column with no postal code
regex_pattern = r'\b[A-Z]\d[A-Z] \d[A-Z]\d\b'
apartments_data_out_df['address_no_postal_code'] = apartments_data_out_df['apartment_address'].str.replace(regex_pattern, '', regex=True).str.strip()

# Display the first few rows of the dataframe
print(f"\n------------ Apartments data df shape: {apartments_data_out_df.shape}")
print("\n------------")
print(apartments_data_out_df.head())


------------ Apartments data df shape: (117, 8)

------------
                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   

                                       phone_numbers    Elementary_School  \
0                         437-973-3626, 416-665-4430          Rockford PS   
1           437-900-6800, 416-665-2100, 437-900-6811          Rockford PS   
2                         416-485-7207, 416-751-3368    Oriole Park Jr PS   
3           888-244-6696, 647-507-4461, 416-487-4461     Davisville Jr PS   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...  Ledbury Park E & MS   

         Middle_School           High_School street_name  \
0    Fisherville Sr PS  Northview Heights SS     Antibes   
1    Fisherville 

In [13]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

geolocator = Nominatim(user_agent="myApp")

In [14]:
ind = 2
address_check = apartments_data_out_df['address_no_postal_code'].iloc[ind]
print(address_check)

location = geolocator.geocode(address_check)
lat = location.latitude
lon = location.longitude
print(lat, lon)

901 Avenue Rd, Toronto, ON
43.700344666666666 -79.40666533333334


In [15]:
print( geolocator.geocode('110 Inverlochy Blvd, Markham, ON') )

110, Inverlochy Boulevard, Langstaff, Markham, York Region, Golden Horseshoe, Ontario, L3T 3C0, Canada


In [16]:
def apply_geocode(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding address: {address} - {e}")
        return (None, None)

In [17]:
tqdm.pandas(desc="Geocoding addresses")
apartments_data_out_df['lat_long'] = apartments_data_out_df['address_no_postal_code'].progress_apply(apply_geocode)

print(apartments_data_out_df.head())

Geocoding addresses: 100%|██████████| 117/117 [01:00<00:00,  1.94it/s]

                          apartment_address num_beds  \
0    155 Antibes Dr, North York, ON M2R 3G7  1, 2, 3   
1    300 Antibes Dr, North York, ON M2R 3N8  1, 2, 3   
2        901 Avenue Rd, Toronto, ON M5P 2K7  1, 2, 3   
3   155 Balliol Street, Toronto, ON M4S 1C4  1, 2, 3   
4  3575 Bathurst St, North York, ON M6A 2C8     1, 2   

                                       phone_numbers    Elementary_School  \
0                         437-973-3626, 416-665-4430          Rockford PS   
1           437-900-6800, 416-665-2100, 437-900-6811          Rockford PS   
2                         416-485-7207, 416-751-3368    Oriole Park Jr PS   
3           888-244-6696, 647-507-4461, 416-487-4461     Davisville Jr PS   
4  647-559-7317, 647-694-8048, 437-227-9562, 647-...  Ledbury Park E & MS   

         Middle_School           High_School street_name  \
0    Fisherville Sr PS  Northview Heights SS     Antibes   
1    Fisherville Sr PS  Northview Heights SS     Antibes   
2           Hodgson 

##### Create a map

In [18]:
import folium

# Create a base map
m = folium.Map(location=[43.75491558522391, -79.4229619534175], zoom_start=10)

# Iterate through the DataFrame and add each lat/long as a marker on the map
for index, row in apartments_data_out_df.iterrows():
    lat, lon = row['lat_long']
    phone_number = row['phone_numbers']
    apartment_address = row['apartment_address']
    beds = row['num_beds']
    elem_school = row['Elementary_School']
    middle_school = row['Middle_School']
    high_school = row['High_School']

    # Create a popup with the desired information
    # popup_text = f"Address: {apartment_address}<br>Bedrooms: {beds}<br>Phone: {phone_number}"
    popup_html = f"""
        <span style="font-size: 14px;"><strong>Address:</strong> {apartment_address}</span><br>
        <span style="font-size: 14px;"><strong>Bedrooms:</strong> {beds}</span><br>
        <span style="font-size: 14px;"><strong>Phone numbers:</strong> {phone_number}</span><br><br>
        <span style="font-size: 14px;"><strong>Elementary School:</strong> {elem_school}</span><br>
        <span style="font-size: 14px;"><strong>Middle School:</strong> {middle_school}</span><br>
        <span style="font-size: 14px;"><strong>High School:</strong> {high_school}</span>
        """
    popup = folium.Popup(popup_html, max_width=300)

    # folium.Marker([lat, lon]).add_to(m)
    folium.Marker([lat, lon], popup=popup).add_to(m)

# Save the map to an HTML file
m.save('my_map.html')

#### Get school addresses

In [ ]:
def find_school_address(address, query_str):
    query = address.replace(' ', '+') + query_str

    url = f"https://www.google.com/search?q={query}"
    url_apartments_com = f"https://www.google.com/search?q={query_apartments_com}"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
query_str = "+school+address"